# Model 2a and 2b: Non-arbitrage Model

1. Model 2a: 49 firm-level characteristics + 8 macro-economic variable 
2. Model 2b: 49 firm-level characteristics + 134 macro-economic variable 

In [5]:

from keras_tuner import HyperModel
import keras_tuner as kt

import numpy as np
import pandas as pd
#!pip install keras_tuner
#import keras_tuner

import keras
from keras import models, layers, metrics, Sequential
from keras.layers import Dense, BatchNormalization, Dropout
import tensorflow as tf
from keras.regularizers import l1,l2,l1_l2
from sklearn.metrics import mean_squared_error
from tensorflow.keras.layers import Input, Dense, Multiply, Concatenate
from tensorflow.keras.models import Model

import tensorflow as tf
import random

import keras_tuner
from keras_tuner import HyperModel
import keras_tuner as kt

from sklearn.preprocessing import MinMaxScaler

In [6]:
# Import Dataset
import pandas as pd
path = 'dataset/'
firm_df = pd.read_pickle(path+'firm_df49.pkl')
firm_df.index = pd.to_datetime(firm_df.index).to_period('M')

# macro-economic variables
# There is one features which has 37 missing values but before year 1980
macro_df = pd.read_pickle(path+'macro_df.pkl')
macro_df.index = pd.to_datetime(macro_df.index, format='%m/%d/%Y')
macro_df.rename(columns={"ep": "ep_macro"}, inplace=True)
macro_index = macro_df.index[1:]
macro_new = macro_df.iloc[:-1, :]
macro_new.index = macro_index
macro_new.index = macro_new.index.to_period('M')

# Scale macro-features
from sklearn.preprocessing import MinMaxScaler
#scaler = MinMaxScaler(feature_range=(-0.5, 0.5))
#macro_scaled = scaler.fit_transform(macro_new)
#macro_scaled_df = pd.DataFrame(macro_scaled, columns=macro_new.columns)
#macro_scaled_df.index = macro_new.index

# merged data 1
merged_df = pd.merge(firm_df, macro_new, left_on='jdate', right_on='sasdate', how='inner')
merged_df.index = firm_df.index

In [7]:
char_core = ['acc','agr','beta','bm','cash','cashpr','cfp','chatoia','chcsho','chfeps','chinv',
       'chmom','chpmia','chtx','currat','depr','dy','ear','ep','gma','grcapx','grltnoa',
       'ill','indmom','invest','lev','lgr','maxret','mom12m','mom1m','mom36m','mve','nincr',
       'orgcap','pchgm_pchsale','pchsale_pchinvt','pchsale_pchrect','pchsale_pchxsga',
       'retvol','roaq','roavol','roeq','salecash','saleinv','sgr','sp','std_dolvol','std_turn','turn']

info_list = ['fyear','year','jyear','permno','ticker','comnam','exchcd','exchname','siccd',
       'indname','size_class','mve_m','rf','ret','ret_adj','ret_ex','ret_adj_ex',]

macro_col = ['RPI', 'W875RX1', 'DPCERA3M086SBEA', 'CMRMTSPLx', 'RETAILx','INDPRO', 'IPFPNSS', 'IPFINAL', 'IPCONGD', 'IPDCONGD', 'IPNCONGD',
       'IPBUSEQ', 'IPMAT', 'IPDMAT', 'IPNMAT', 'IPMANSICS', 'IPB51222S','IPFUELS', 'CUMFNS', 'HWI', 'HWIURATIO', 'CLF16OV', 'CE16OV',
       'UNRATE', 'UEMPMEAN', 'UEMPLT5', 'UEMP5TO14', 'UEMP15OV','UEMP15T26', 'UEMP27OV', 'CLAIMSx', 'PAYEMS', 'USGOOD',
       'CES1021000001', 'USCONS', 'MANEMP', 'DMANEMP', 'NDMANEMP','SRVPRD', 'USTPU', 'USWTRADE', 'USTRADE', 'USFIRE', 'USGOVT',
       'CES0600000007', 'AWOTMAN', 'AWHMAN', 'HOUST', 'HOUSTNE','HOUSTMW', 'HOUSTS', 'HOUSTW', 'PERMIT', 'PERMITNE', 'PERMITMW',
       'PERMITS', 'PERMITW', 'AMDMNOx', 'ANDENOx', 'AMDMUOx', 'BUSINVx','ISRATIOx', 'M1SL', 'M2SL', 'M2REAL', 'BOGMBASE', 'TOTRESNS',
       'NONBORRES', 'BUSLOANS', 'REALLN', 'NONREVSL', 'CONSPI', 'S&P 500','S&P: indust', 'S&P div yield', 'S&P PE ratio', 'FEDFUNDS',
       'CP3Mx', 'TB3MS', 'TB6MS', 'GS1', 'GS5', 'GS10', 'AAA', 'BAA','COMPAPFFx', 'TB3SMFFM', 'TB6SMFFM', 'T1YFFM', 'T5YFFM', 'T10YFFM',
       'AAAFFM', 'BAAFFM', 'TWEXAFEGSMTHx', 'EXSZUSx', 'EXJPUSx','EXUSUKx', 'EXCAUSx', 'WPSFD49207', 'WPSFD49502', 'WPSID61',
       'WPSID62', 'OILPRICEx', 'PPICMM', 'CPIAUCSL', 'CPIAPPSL','CPITRNSL', 'CPIMEDSL', 'CUSR0000SAC', 'CUSR0000SAD',
       'CUSR0000SAS', 'CPIULFSL', 'CUSR0000SA0L2', 'CUSR0000SA0L5','PCEPI', 'DDURRG3M086SBEA', 'DNDGRG3M086SBEA', 'DSERRG3M086SBEA',
       'CES0600000008', 'CES2000000008', 'CES3000000008', 'UMCSENTx','DTCOLNVHFNM', 'DTCTHFNM', 'INVEST', 'VIXCLSx', 'dp', 'ep_macro', 'b/m',
       'ntis', 'tbl', 'tms', 'dfy', 'svar']

macro_core =  ['dp', 'ep_macro', 'b/m','ntis', 'tbl', 'tms', 'dfy', 'svar']

def datasplit(df, dataset):
    df_train = df[df.index < pd.Period((str(1995)+"-1"),freq='M')]
    df_val = df[(df.index >= pd.Period((str(1995)+"-1"),freq='M')) & (df.index < pd.Period((str(2000)+"-1"),freq='M'))]
    df_test = df[df.index >= pd.Period((str(2000)+"-1"),freq='M')]
    if dataset == "firm":
        X_train = df_train[char_core]
        X_val = df_val[char_core]
        X_test = df_test[char_core]
    elif dataset == 'macro':
        X_train = df_train[macro_col]
        X_val = df_val[macro_col]
        X_test = df_test[macro_col]
    elif dataset == "firm_macro":
        X_train = df_train[char_core+macro_col]
        X_val = df_val[char_core+macro_col]
        X_test = df_test[char_core+macro_col]
    elif dataset == "firm_macro8":
        X_train = df_train[char_core+macro_core]
        X_val = df_val[char_core+macro_core]
        X_test = df_test[char_core+macro_core] 
    y_train = df_train['predicted_return']
    y_val = df_val['predicted_return']
    y_test = df_test['predicted_return']
    return (X_train, y_train, X_val, y_val, X_test, y_test, )

In [95]:
from keras.layers import Dropout

def my_metric_fn(y_true, y_pred):
    num = tf.reduce_mean(tf.square(y_true - y_pred))
    den = tf.reduce_mean(tf.square(y_true - tf.zeros_like(y_true)))
    return 1 - num / den

def call_existing_code(input_value):
    L = 3
    l1_reg= 0.
    l2_reg= 0.1
    dropout_ =0.2
    lr =0.0001
    acti='relu'
    model = Sequential()
    model.add(Dense(units=64, input_dim=input_value, kernel_regularizer=tf.keras.regularizers.l1_l2(l1=l1_reg, l2=l2_reg), activation=acti))
    model.add(Dropout(dropout_))
    model.add(Dense(units=32, kernel_regularizer=tf.keras.regularizers.l1_l2(l1=l1_reg, l2=l2_reg), activation=acti))
    model.add(Dropout(dropout_))
    model.add(Dense(units=16, kernel_regularizer=tf.keras.regularizers.l1_l2(l1=l1_reg, l2=l2_reg), activation=acti))
    model.add(Dropout(dropout_))
    model.add(Dense(units=8, kernel_regularizer=tf.keras.regularizers.l1_l2(l1=l1_reg, l2=l2_reg), activation=acti))
    model.add(Dropout(dropout_))
    optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=lr)
    model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=[my_metric_fn])
    return model

In [8]:
def my_metric_fn(y_true, y_pred):
    num = tf.reduce_mean(tf.square(y_true - y_pred))
    den = tf.reduce_mean(tf.square(y_true))
    return 1 - num / den

class MyHyperModel(keras_tuner.HyperModel):
    def __init__(self,num_input):
        self.num_input = num_input

    def build(self, hp,):
        l1_reg = hp.Choice("l1_ratio", [0.])
        l2_reg = hp.Choice("l2_ratio", [0.001])
        init_mode = 'glorot_normal'
        layer = hp.Choice("layer", [4])
        acti = 'relu'
        dropout = hp.Choice("dropout",[0.1,])
        lr = hp.Choice("learning_rate", [0.0001])
        model = keras.Sequential()
        model.add(Dense(units=64, input_dim=self.num_input, kernel_regularizer=l1_l2(l1=l1_reg, l2=l2_reg), kernel_initializer=init_mode, activation=acti))
        model.add(Dropout(dropout))
        model.add(BatchNormalization())
        for i in range (layer-1):
            model.add(Dense(int(32/2**i),kernel_regularizer=l1_l2(l1_reg,l2=l2_reg), kernel_initializer=init_mode, activation=acti))
            model.add(Dropout(dropout))
            model.add(BatchNormalization())
        model.add(Dense(1, kernel_initializer=init_mode)) 
        opt = tf.keras.optimizers.legacy.Adam(learning_rate=lr)
        model.compile(loss='mean_squared_error', optimizer=opt,metrics=[my_metric_fn])
        return model
    
    def fit(self, hp, model, *args, **kwargs):
        return model.fit(
            *args,
            batch_size=hp.Choice("batch_size", [256]),
            **kwargs,)

## 49 Firm-level

In [9]:
X_train, y_train, X_val, y_val, X_test, y_test = datasplit(firm_df, dataset = "firm")
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape)

(88347, 49) (88347,) (31851, 49) (31851,) (90007, 49) (90007,)


In [10]:
tuner = keras_tuner.RandomSearch(MyHyperModel(X_train.shape[1]),objective='val_loss',max_trials=10,overwrite=True,)
tuner.search(X_train, y_train, epochs=50, 
             validation_data=(X_val, y_val), 
             callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)])

Trial 1 Complete [00h 00m 20s]
val_loss: 0.014586332254111767

Best val_loss So Far: 0.014586332254111767
Total elapsed time: 00h 00m 20s
INFO:tensorflow:Oracle triggered exit


In [ ]:
# 10 Trials for Prediction
x_all = np.concatenate((X_train, X_val))
y_all = np.concatenate((y_train, y_val))
seed_list = [458, 165, 530, 564, 590, 560, 829, 170, 376, 176]
yhat_df = pd.DataFrame()

for random_seed in seed_list:
    random.seed(random_seed)
    np.random.seed(random_seed)
    tf.random.set_seed(random_seed)

    best_model = tuner.get_best_models(num_models=1)[0]
    best_model.fit(x=x_all, y=y_all, epochs=20,verbose=0,
              callbacks=[keras.callbacks.EarlyStopping(monitor='loss',mode='min', patience=2)])
    print(best_model.evaluate(X_test,y_test))
    y_hat = best_model.predict(X_test).reshape(-1)
    print(y_hat.mean(),y_hat.std())
    yhat_df[random_seed] = y_hat
    print()

In [15]:
# Print out Predictive R^2
y_predict = yhat_df.mean(axis=1).values.reshape(-1)
y_real = y_test
a = np.mean(np.square(y_predict -  y_real))
b = np.mean(np.square(y_real))
R2 = 1-a/b
print(f"R^2 is {R2}")

path_y = 'predict/'
yhat_df.to_pickle(path_y+"model2c49.pkl")

R^2 is 0.006558914378433256


## 49 Firm-level + 8 Macroeconomic

In [17]:
X_train, y_train, X_val, y_val, X_test, y_test = datasplit(merged_df, dataset = "firm_macro8")
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape)

(88347, 57) (88347,) (31851, 57) (31851,) (90007, 57) (90007,)


In [18]:
def my_metric_fn(y_true, y_pred):
    num = tf.reduce_mean(tf.square(y_true - y_pred))
    den = tf.reduce_mean(tf.square(y_true))
    return 1 - num / den

class MyHyperModel(keras_tuner.HyperModel):
    def __init__(self,num_input):
        self.num_input = num_input

    def build(self, hp,):
        l1_reg = hp.Choice("l1_ratio", [0.])
        l2_reg = hp.Choice("l2_ratio", [0.001,0.])
        init_mode = 'glorot_normal'
        layer = hp.Choice("layer", [4])
        acti = 'relu'
        dropout = hp.Choice("dropout",[0.1,0.2])
        lr = hp.Choice("learning_rate", [0.0001])
        model = keras.Sequential()
        model.add(Dense(units=64, input_dim=self.num_input, kernel_regularizer=l1_l2(l1=l1_reg, l2=l2_reg), kernel_initializer=init_mode, activation=acti))
        model.add(Dropout(dropout))
        model.add(BatchNormalization())
        for i in range (layer-1):
            model.add(Dense(int(32/2**i),kernel_regularizer=l1_l2(l1_reg,l2=l2_reg), kernel_initializer=init_mode, activation=acti))
            model.add(Dropout(dropout))
            model.add(BatchNormalization())
        model.add(Dense(1, kernel_initializer=init_mode)) 
        opt = tf.keras.optimizers.legacy.Adam(learning_rate=lr)
        model.compile(loss='mean_squared_error', optimizer=opt,metrics=[my_metric_fn])
        return model
    
    def fit(self, hp, model, *args, **kwargs):
        return model.fit(
            *args,
            batch_size=hp.Choice("batch_size", [256,128]),
            **kwargs,)

In [19]:
tuner = keras_tuner.RandomSearch(MyHyperModel(X_train.shape[1]),objective='val_loss',max_trials=10,overwrite=True,)
tuner.search(X_train, y_train, epochs=50, 
             validation_data=(X_val, y_val), 
             callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)])

Trial 8 Complete [00h 00m 12s]
val_loss: 0.01454280223697424

Best val_loss So Far: 0.014537124894559383
Total elapsed time: 00h 02m 24s
INFO:tensorflow:Oracle triggered exit


In [ ]:
# 10 Trials for Prediction
x_all = np.concatenate((X_train, X_val))
y_all = np.concatenate((y_train, y_val))
seed_list = [458, 165, 530, 564, 590, 560, 829, 170, 376, 176]
yhat_df = pd.DataFrame()

for random_seed in seed_list:
    random.seed(random_seed)
    np.random.seed(random_seed)
    tf.random.set_seed(random_seed)

    best_model = tuner.get_best_models(num_models=1)[0]
    best_model.fit(x=x_all, y=y_all, epochs=20,verbose=0,
              callbacks=[keras.callbacks.EarlyStopping(monitor='loss',mode='min', patience=2)])
    print(best_model.evaluate(X_test,y_test))
    y_hat = best_model.predict(X_test).reshape(-1)
    print(y_hat.mean(),y_hat.std())
    yhat_df[random_seed] = y_hat
    print()

In [22]:
# Print out Predictive R^2
y_predict = yhat_df.mean(axis=1).values.reshape(-1)
y_real = y_test
a = np.mean(np.square(y_predict -  y_real))
b = np.mean(np.square(y_real))
R2 = 1-a/b
print(f"R^2 is {R2}")

path_y = 'predict/'
yhat_df.to_pickle(path_y+"model2a49_8.pkl")

R^2 is 0.001230689943262031


## 49 Firm-level + 134 Macro

In [24]:
X_train, y_train, X_val, y_val, X_test, y_test = datasplit(merged_df, dataset = "firm_macro")
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape)

(88347, 183) (88347,) (31851, 183) (31851,) (90007, 183) (90007,)


In [25]:
def my_metric_fn(y_true, y_pred):
    num = tf.reduce_mean(tf.square(y_true - y_pred))
    den = tf.reduce_mean(tf.square(y_true))
    return 1 - num / den

class MyHyperModel(keras_tuner.HyperModel):
    def __init__(self,num_input):
        self.num_input = num_input

    def build(self, hp,):
        l1_reg = hp.Choice("l1_ratio", [0.])
        l2_reg = hp.Choice("l2_ratio", [0.001,0.])
        init_mode = 'glorot_normal'
        layer = hp.Choice("layer", [4])
        acti = 'relu'
        dropout = hp.Choice("dropout",[0.1,0.2])
        lr = hp.Choice("learning_rate", [0.0001])
        model = keras.Sequential()
        model.add(Dense(units=64, input_dim=self.num_input, kernel_regularizer=l1_l2(l1=l1_reg, l2=l2_reg), kernel_initializer=init_mode, activation=acti))
        model.add(Dropout(dropout))
        model.add(BatchNormalization())
        for i in range (layer-1):
            model.add(Dense(int(32/2**i),kernel_regularizer=l1_l2(l1_reg,l2=l2_reg), kernel_initializer=init_mode, activation=acti))
            model.add(Dropout(dropout))
            model.add(BatchNormalization())
        model.add(Dense(1, kernel_initializer=init_mode)) 
        opt = tf.keras.optimizers.legacy.Adam(learning_rate=lr)
        model.compile(loss='mean_squared_error', optimizer=opt,metrics=[my_metric_fn])
        return model
    
    def fit(self, hp, model, *args, **kwargs):
        return model.fit(
            *args,
            batch_size=hp.Choice("batch_size", [256,128]),
            **kwargs,)

In [26]:
tuner = keras_tuner.RandomSearch(MyHyperModel(X_train.shape[1]),objective='val_loss',max_trials=10,overwrite=True,)
tuner.search(X_train, y_train, epochs=50, 
             validation_data=(X_val, y_val), 
             callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)])

Trial 8 Complete [00h 00m 11s]
val_loss: 0.01448238268494606

Best val_loss So Far: 0.01448238268494606
Total elapsed time: 00h 02m 26s
INFO:tensorflow:Oracle triggered exit


In [ ]:
# 10 Trials for Prediction
x_all = np.concatenate((X_train, X_val))
y_all = np.concatenate((y_train, y_val))
seed_list = [458, 165, 530, 564, 590, 560, 829, 170, 376, 176]
yhat_df = pd.DataFrame()

for random_seed in seed_list:
    random.seed(random_seed)
    np.random.seed(random_seed)
    tf.random.set_seed(random_seed)

    best_model = tuner.get_best_models(num_models=1)[0]
    best_model.fit(x=x_all, y=y_all, epochs=20,verbose=0,
              callbacks=[keras.callbacks.EarlyStopping(monitor='loss',mode='min', patience=2)])
    print(best_model.evaluate(X_test,y_test))
    y_hat = best_model.predict(X_test).reshape(-1)
    print(y_hat.mean(),y_hat.std())
    yhat_df[random_seed] = y_hat
    print()

In [30]:
# Print out Predictive R^2
y_predict = yhat_df.mean(axis=1).values.reshape(-1)
y_real = y_test
a = np.mean(np.square(y_predict -  y_real))
b = np.mean(np.square(y_real))
R2 = 1-a/b
print(f"R^2 is {R2}")

path_y = 'predict/'
yhat_df.to_pickle(path_y+"model2b49_134.pkl")

R^2 is -1.8604247827026152
